<a href="https://colab.research.google.com/github/zixian2021/Zhenhuan-VITS-fast-fine-tuning/blob/main/Zhenhuan_VITS_fast_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#STEP 0  
##本笔记本必须在GPU模式下运行，可以运行上方的nvidia-smi查看GPU是否装载。若没有，点击左上角>修改>笔记本设置，把硬件加速改成GPU。  
##！！！开始之前请仔细阅读[数据集上传格式要求](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA.MD)避免后续因为读取不到音频文件而报错。！！！
##Note: GPU is necessary for this notebook. Check whether your GPU is loaded before you start.Please read [Requirements for uploaded data format](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA_EN.MD) carefully to avoid any possible error caused by voice file not found.


In [ ]:
# 查看GPU配置
# Check GPU configuration
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
#@title STEP 1 复制代码库并安装运行环境
#@markdown #STEP 1 (3 min)
#@markdown ##复制代码库并安装运行环境
#@markdown ##Clone repository & Build environment
! git clone https://github.com/zixian2021/Zhenhuan-VITS-fast-fine-tuning.git
!pip install imageio==2.4.1
!pip install --upgrade youtube-dl
!pip install moviepy
!pip install git+https://github.com/openai/whisper.git
!python3 -m pip install -U demucs

%cd /content/Zhenhuan-VITS-fast-fine-tuning
!pip install -r requirements.txt

# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
!cp -r /content/Zhenhuan-VITS-fast-fine-tuning/whisper/transcribe.py /usr/local/lib/python3.9/dist-packages/whisper
!cp -r /content/Zhenhuan-VITS-fast-fine-tuning/whisper/utils.py /usr/local/lib/python3.9/dist-packages/whisper
import whisper
# download data for fine-tuning
#!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
#!unzip sampled_audio4ft_v2.zip
# create necessary directories
!mkdir video_data
!mkdir raw_audio
!mkdir separated_data
!mkdir denoised_audio
!mkdir custom_character_voice
!mkdir segmented_character_voice

Cloning into 'Zhenhuan-VITS-fast-fine-tuning'...
remote: Enumerating objects: 630, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 630 (delta 224), reused 214 (delta 198), pack-reused 351
Receiving objects: 100% (630/630), 2.45 MiB | 14.33 MiB/s, done.
Resolving deltas: 100% (344/344), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=bb92d5ed51d1a4999263355b970110fdadcd7fdca5ab641ddf6c90393c16cdc3
  Stored in directory: /root/.cache/pip/wheels/b7/44/b7/2e7cc9c5fe4a893b9cc83a010d4410557bedf6cf3b5829f497
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.25.1
    Uninstalling imageio-2.25.1:
      Successfully

In [ ]:
#@title （可选）加载Google云端硬盘 / Mount Google drive
#@title (optional) 

#@markdown 加载Google云端硬盘（更快地上传数据集文件）  

#@markdown Mount Google drive for faster data upload
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

## Step2 数据准备

###STEP 2.1 上传短音频（单条时长应该为
### Short audio upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run voice_upload.py --type zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

Saving test.wav to test.wav
Archive:  ./custom_character_voice/custom_character_voice.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./custom_character_voice/custom_character_voice.zip or
        ./custom_character_voice/custom_character_voice.zip.zip, and cannot find ./custom_character_voice/custom_character_voice.zip.ZIP, period.


# step2.1 音频数据上传,支持多人对讲

上传长音频 （单个不应长于20分钟）
Long audio upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%cd /content/Zhenhuan-VITS-fast-fine-tuning/
%run voice_upload.py --type audio

/content/Zhenhuan-VITS-fast-fine-tuning


In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有长音频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径：  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the long audios under one folder, and fill in the path to your folder below:
AUDIO_FOLDER_PATH = "../drive/MyDrive/long_audios/"  #@param {type:"string"}
!cp {AUDIO_FOLDER_PATH}/* ./raw_audio/

cp: cannot stat '../drive/MyDrive/long_audios//*': No such file or directory


In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将`.zip`文件上传云端硬盘后，在下面填写文件路径：  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. After uploading your `.zip` file to Google drive, fill in the path to your file below:
ZIP_PATH = "../drive/MyDrive/samples.zip"  #@param {type:"string"}
!cp {ZIP_PATH} ./custom_character_voice/custom_character_voice.zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

cp: cannot stat '../drive/MyDrive/samples.zip': No such file or directory
Archive:  ./custom_character_voice/custom_character_voice.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./custom_character_voice/custom_character_voice.zip or
        ./custom_character_voice/custom_character_voice.zip.zip, and cannot find ./custom_character_voice/custom_character_voice.zip.ZIP, period.


### STEP 2.3 上传视频（单个不应长于20分钟）
### Video upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run voice_upload.py --type video

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有视频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径:  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the videos under one folder, and fill in the path to your folder below:
VIDEO_FOLDER_PATH = "../drive/MyDrive/videos/"  #@param {type:"string"}
!cp {VIDEO_FOLDER_PATH}/* ./video_data/

cp: cannot stat '../drive/MyDrive/videos//*': No such file or directory


# step2.2 音频数据切分，切分后的数据放在separated_data文件夹

In [ ]:
#根据静音将长语音段切分为短句
!pip install pydub
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence




for file in os.listdir('./raw_audio/'):
  
  if file.endswith('.wav'):
    index = file.split('.')[0]
    file_path = './raw_audio/' + file
    #print(file_path)

    sound = AudioSegment.from_mp3(file_path)
    loudness = sound.dBFS
    #print(loudness)
 
    chunks = split_on_silence(sound,
    # must be silent for at least half a second,沉默半秒
    min_silence_len=430,
 
    # consider it silent if quieter than -16 dBFS
    silence_thresh=-45,
    keep_silence=400
 
    )
    print('总分段：', len(chunks))
 
    # 放弃长度小于2秒的录音片段
    for i in list(range(len(chunks)))[::-1]:
      if len(chunks[i]) <= 2000:
        chunks.pop(i)
    print('取有效分段(大于2s)：', len(chunks))
 
    for i, chunk in enumerate(chunks):
      chunk.export("./separated_data/{0}_chunk{1}.wav".format(index,i), format="wav")
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
总分段： 7
取有效分段(大于2s)： 5


# step2.3 分离出音频中的纯净人声，存放在vocal_data文件夹

In [ ]:
#demucs 分离音轨，生成纯语音数据
!mkdir ./vocal_data 
!mkdir ./demucs_data

import os
import torchaudio
raw_audio_dir = "./separated_data/"
vocal_dir = "./vocal_data/"

for file in os.listdir(raw_audio_dir):

    if file.endswith(".wav"):
        os.system(f"demucs -j 8 --two-stems=vocals {raw_audio_dir}{file} -o ./demucs_data")


for file in os.listdir(raw_audio_dir):
  file = file.replace(".wav","")
  wav, sr = torchaudio.load(f"./demucs_data/htdemucs/{file}/vocals.wav",frame_offset=0,num_frames=-1,normalize=True,
                            channels_first=True)
  wav = wav.mean(dim=0).unsqueeze(0)
  if(sr!=22050):
    wav = torchaudio.transforms.Resample(orig_freq=sr, new_freq=22050)(wav)

  torchaudio.save(vocal_dir+file+'.wav', wav, 22050, channels_first=True)



# step2.4 声纹识别，将分段的人声对应到各个说话人，存放于selected_character_wav


In [10]:
%cd /content
!git clone https://github.com/zixian2021/Voiceprint.git
%cd /content/Voiceprint/

!python -m pip install mvector -U -i https://pypi.tuna.tsinghua.edu.cn/simple
!python setup.py install

/content
Cloning into 'Voiceprint'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 61 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (61/61), 1.09 MiB | 4.08 MiB/s, done.
/content/Voiceprint
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5b/6e/0fccaa0ee098b235176479c0a670c34bd022c1056cd75246f7096086ff16/mvector-0.3.7-py3-none-any.whl (53 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/45/65/46c870a909c65f81f2e59e4501e47685650bc31e6c3483ee23b5a42a4538/visualdl-2.5.1-py3-none-any.whl (6.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c9/f7/7574f62f4fe299155f85b327afa24d20fc22c3e6dd25ad121a7db959f79f/flask_babel-3.0.1-py3-none-any.whl (11 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/49/47/1a47ff

# 重要：需要下载configs文件夹和model文件夹，手动放在voiceprint文件夹内：

config：https://drive.google.com/drive/folders/14gJNd6Wty1Cm8Nca2QiYkYrzjUikb0W4?usp=share_link

models：https://drive.google.com/drive/folders/1onxuAsj2C_qIRLypISn0v0n7XJ5TYJQR?usp=share_link

In [19]:

#提取分段后干净音频中对应说话人的音频
import os
import argparse
import functools
import torch
from mvector.predict import MVectorPredictor
from mvector.utils.utils import add_arguments, print_arguments

%cd /content/Voiceprint/



# !cp -r /content/drive/MyDrive/VoiceprintRecognition_Pytorch/configs ./
# !cp -r /content/drive/MyDrive/VoiceprintRecognition_Pytorch/models ./


predictor = MVectorPredictor(configs='configs/ecapa_tdnn.yml',
                             model_path='models/ecapa_tdnn_MelSpectrogram/best_model/model.pt',
                             use_gpu=False)

%cd /content/Zhenhuan-VITS-fast-fine-tuning/
!mkdir ./selected_character_wav
#设定判定门限值，可调整
threshold = 0.6

#第一个说话人 甄嬛
speaker_id = 0
for file in os.listdir('/content/Zhenhuan-VITS-fast-fine-tuning/vocal_data/'):

    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/zhenhuan.wav' 
    
    file_wav = './vocal_data/' + file
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))
      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))

#第二个说话人 皇上
speaker_id = 1
for file in os.listdir('./vocal_data/'):
    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/silang.wav' 
    file_wav = './vocal_data/' + file
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))

      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))

speaker_id = 2

for file in os.listdir('./vocal_data/'):
    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/huafei.wav' 
    file_wav = './vocal_data/' + file
    # target = '/content/drive/MyDrive/character_wav/' + str(speaker_id)+ '_' + file
    # print(target)
    print(file_wav)
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))
      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))

speaker_id = 3
for file in os.listdir('./vocal_data/'):
    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/huanghou.wav' 
    file_wav = './vocal_data/' + file
    # target = '/content/drive/MyDrive/character_wav/' + str(speaker_id)+ '_' + file
    # print(target)
    print(file_wav)
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))

      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))


speaker_id = 4
for file in os.listdir('./vocal_data/'):
    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/meizhuang.wav' 
    file_wav = './vocal_data/' + file
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))

      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))


speaker_id = 5
for file in os.listdir('./vocal_data/'):
    speaker_wav = '/content/Zhenhuan-VITS-fast-fine-tuning/character/xiaoniao.wav' 
    file_wav = './vocal_data/' + file
    # target = '/content/drive/MyDrive/character_wav/' + str(speaker_id)+ '_' + file
    # print(target)
    # print(file_wav)
    
    dist = predictor.contrast(speaker_wav, file_wav)
    if dist > threshold:
      print('recognize as'+str(speaker_id))

      os.system('mv {0} {1}'.format(file_wav, './selected_character_wav/'))



/content/Voiceprint
[2023-04-02 04:31:33.702398 INFO   ] utils:print_arguments:18 - ----------- 配置文件参数 -----------
[2023-04-02 04:31:33.704004 INFO   ] utils:print_arguments:21 - dataset_conf:
[2023-04-02 04:31:33.706133 INFO   ] utils:print_arguments:28 - 	batch_size: 64
[2023-04-02 04:31:33.707761 INFO   ] utils:print_arguments:28 - 	chunk_duration: 3
[2023-04-02 04:31:33.710443 INFO   ] utils:print_arguments:28 - 	do_vad: False
[2023-04-02 04:31:33.711770 INFO   ] utils:print_arguments:28 - 	label_list_path: dataset/label_list.txt
[2023-04-02 04:31:33.714803 INFO   ] utils:print_arguments:28 - 	min_duration: 0.5
[2023-04-02 04:31:33.716120 INFO   ] utils:print_arguments:28 - 	num_speakers: 3242
[2023-04-02 04:31:33.718102 INFO   ] utils:print_arguments:28 - 	num_workers: 4
[2023-04-02 04:31:33.719795 INFO   ] utils:print_arguments:28 - 	sample_rate: 16000
[2023-04-02 04:31:33.720656 INFO   ] utils:print_arguments:28 - 	target_dB: -20
[2023-04-02 04:31:33.721740 INFO   ] utils:print_

/usr/local/lib/python3.9/dist-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (513) may be set too low.
  warnings.warn(


recognize as0
recognize as0
./vocal_data/test_chunk4.wav
./vocal_data/test_chunk2.wav
./vocal_data/test_chunk1.wav
./vocal_data/test_chunk4.wav
./vocal_data/test_chunk2.wav
./vocal_data/test_chunk1.wav
recognize as5


In [ ]:
#@title STEP 1.5 选择预训练模型
#@markdown ###STEP 1.5 选择预训练模型  
#@markdown ###Choose pretrained model to start  
#@markdown CJE为中日英三语模型，CJ为中日双语模型  

#@markdown CJE for Chinese, Japanese & English model，CJ for Chinese & Japanese model
# PRETRAINED_MODEL = "CJ" #@param ["CJE","CJ"]
# if PRETRAINED_MODEL == "CJ":
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0-p.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/config.json -O ./configs/finetune_speaker.json
# elif PRETRAINED_MODEL == "CJE":
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/configs/uma_trilingual.json -O ./configs/finetune_speaker.json
# elif PRETRAINED_MODEL == "C": # Bad generalizability to other voices
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-bb/D_0.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-bb/G_0.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-bb/config.json -O ./configs/finetune_speaker.json

## STEP 3 语音识别挑选出的数据

In [20]:
#@markdown 运行该单元格会对所有上传的数据进行自动去背景音&标注。 
#@markdown 由于需要调用Whisper和Demucs，运行时间可能较长。  

#@markdown Running this codeblock will perform automatic vocal seperation & annotation. 
#@markdown Since this step uses Whisper & Demucs, it may take a while to complete.
# 将所有视频（无论是上传的还是下载的，且必须是.mp4格式）抽取音频
%cd /content/drive/MyDrive/Zhenhuan-VITS-fast-fine-tuning
#%run video2audio.py
# 将所有音频（无论是上传的还是从视频抽取的，必须是.wav格式）去噪
#!python denoise_audio.py
# 分割并标注长音频
!python long_audio_transcribe.py --languages C --whisper_size large
# 标注短音频
#!python short_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large

[Errno 2] No such file or directory: '/content/drive/MyDrive/Zhenhuan-VITS-fast-fine-tuning'
/content/Zhenhuan-VITS-fast-fine-tuning
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:

#！！！训练质量相关：实验发现目前使用CJ模型+勾选ADD_AUXILIARY，对于中/日均能训练出最好的效果，第一次训练建议默认使用该组合！！！

In [ ]:
#@markdown ##STEP 3.5 


#@markdown 运行该单元格会生成划分好训练/测试集的最终标注，以及配置文件  

#@markdown Running this block will generate final annotations for training & validation, as well as config file.

#@markdown 选择是否加入辅助训练数据：/ Choose whether to add auxiliary data: 
ADD_AUXILIARY = True #@param {type:"boolean"}
#@markdown 辅助训练数据是从预训练的大数据集抽样得到的，作用在于防止模型在标注不准确的数据上形成错误映射。

#@markdown Auxiliary data is to prevent overfitting when the audio samples are small or with low quality. 

#@markdown 以下情况请勾选：  

#@markdown 总样本少于100条/样本包含角色只有1人/样本质量一般或较差/样本来自爬取的视频  

#@markdown 以下情况可以不勾选：  

#@markdown 总样本量很大/样本质量很高/希望加速训练/只有二次元角色

#@markdown 样本仅包含单说话人会导致报错，请勾选ADD_AUXILIARY或加入几个dummy speaker绕过去  

#@markdown Feeding voice samples containing single speaker will result in error. Please select ADD_AUXILIARY or add several dummy speakers to bypass.

# assert(not (ADD_AUXILIARY and PRETRAINED_MODEL != "CJE")), "add auxiliary data is available only available for CJE model!"
if ADD_AUXILIARY:
  %run preprocess_v2.py --add_auxiliary_data True --languages C
else:
  %run preprocess_v2.py --languages "{PRETRAINED_MODEL}"

## STEP 4 开始训练

In [ ]:
#@markdown #STEP 4 (>=20 min)
#@markdown 开始微调模型，在40轮训练后会自动结束。
#@markdown 训练时长取决于你录入/上传的音频总数。

#@markdown 根据声线和样本质量的不同，所需的训练epochs数也不同。但是一般建议设置为40 epochs，  

#@markdown 你也可以在Tensorboard中预览合成效果，若效果满意可提前停止。  

#@markdown Model fine-tuning ends after 40 epochs. 
#@markdown Total time cost depends on the number of voices you recorded/uploaded.   

#@markdown Best epoch number varies depending on different uploaded voices / sample quality. Normally, 40 epochs is suggested.

#@markdown You can also preview synthezied audio in Tensorboard, it's OK to shut down training manually if you find the quality is satisfying.
%cd /content/Zhenhuan-VITS-fast-fine-tuning

%reload_ext tensorboard
%tensorboard --logdir "./OUTPUT_MODEL"
Maximum_epochs = "400" #@param [20, 30, 40, 50, 60]
!python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed True

In [ ]:
#@markdown ### 微调完成后，在这里尝试效果。
#@markdown ### Try out TTS & VC quality here after fine-tuning is finished.
%cd /content/drive/MyDrive/VITS-fast-fine-tuning

!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
%run VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth

# STEP 5 下载模型
## 本地部署方法请见[README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)

In [ ]:
#@markdown ### 下载选项1：运行该单元格，浏览器会自动下载模型和配置文件
#@markdown ### Download option 1: Running this codeblock will download model & config files by your browser.
!python rearrange_speaker.py 
%run download_model.py

In [ ]:
#@markdown ### 下载选项2：运行该单元格会将模型和配置文件保存到Google云端硬盘
#@markdown ### Download option 2: Running this codeblock will save the mode & config files to your Google drive.
!python rearrange_speaker.py 
!cp ./G_latest.pth ../drive/MyDrive/G_latest.pth
!cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
!cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

In [ ]:
#@markdown ### 运行该单元格会清空所有已上传的样本，需要时可使用
#@markdown ### Running this codeblock will delete all voice samples you have uploaded. Use it if you need.
!rm -rf ./custom_character_voice/*
!rm -rf ./video_data/*
!rm -rf ./raw_audio/*
!rm -rf ./denoised_audio/*
!rm -rf ./segmented_character_voice/*
!rm -rf long_character_anno.txt
!rm -rf short_character_anno.txt

In [ ]:
#@markdown ### 运行该单元格会将切片和标注复制到谷歌云端硬盘根目录下名为`voice_data`的文件夹下以用作其它用途
#@markdown ### Running this codeblock will copy all processed voices & annotations to a folder named `voice_data` under the root of Google Drive for other purpose of usage
!mkdir ../drive/MyDrive/voice_data/
!cp -rf ./custom_character_voice/ ../drive/MyDrive/voice_data/custom_character_voice/
!cp -rf ./segmented_character_voice/ ../drive/MyDrive/voice_data/segmented_character_voice/
!cp long_character_anno.txt ../drive/MyDrive/voice_data/long_character_anno.txt
!cp short_character_anno.txt ../drive/MyDrive/voice_data/short_character_anno.txt